In [1]:
from glob import glob
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import os
import cv2
import csv
import json
import time
import shutil
import random

from sklearn.model_selection import train_test_split

In [2]:
root_path = "/home/ubuntu/workspace/seunghoon/z_dataset"
target_folder_name = "diablo2_dataset_ver2"

src_folder_name = "labelme_dataset_processing3"
dst_folder_name = "labelme_dataset_for_recognition"

In [3]:
src_imgs = glob(os.path.join(root_path, target_folder_name, src_folder_name, "*/*"))
src_imgs = [i for i in src_imgs if "png" in i or "jpg" in i and not ".json" in i]
print(len(src_imgs))

22886


In [4]:
# make recognition dataset

# error_char는 오류가 나는 단어들을 제외해야 할 경우에 사용합니다.
# 기본적인 특수문자는 남기는 error_char
# error_char = 'ㄷㅇ#.｜°·×˚Ι‧※₩Ⅱ→⇒∙⌜⌟①②③④⑤│■□▣▶◀◆◇◈○●★☆☎☞♡「」『』】ㄴㄹㅁㅏㅔㅗㅣㆍ㎡休元光內前國壬大奉奠子孔家寅年情愛故期木歌死氏海無生祝福秋紀美行辛釋金'
# 현재(5.10)는 error_char를 알 수 없으므로 설정하지 않겠습니다.
error_char = ''

# labelme에서 labeling 작업이 끝난 결과물에서 bbox단위로 crop한 이미지 저장
dst_dir_path = os.path.join(root_path, target_folder_name,  dst_folder_name)
os.makedirs(os.path.join(dst_dir_path, "imgs_src"), exist_ok=True)

cnt = 0
data_lst = []
for img in tqdm(src_imgs):
    json_path = img.replace(os.path.splitext(img)[1], ".json")
    with open(json_path, "r")as f:
        data = json.load(f)
        
    img_ext = os.path.splitext(img)[-1]
    
    im = Image.open(img).convert("RGBA")
    
    for i in data['shapes']:
        # if os.path.isfile(os.path.join(dst_dir_path, "imgs_src", str(cnt).zfill(12) + img_ext)):
        #     continue
        if len(i['label']) == 1:
            continue
        tmp_data = []
        polygon = []
        x_lst, y_lst = [], []
        for pt in i['points']:
            polygon.append((int(pt[0]), int(pt[1])))
            x_lst.append(int(pt[0]))
            y_lst.append(int(pt[1]))
        
        # convert image to numpy (for convenience)
        imArray = np.asarray(im)
                         
        # create mask
        maskIm = Image.new('L', (imArray.shape[1], imArray.shape[0]), 0)
        ImageDraw.Draw(maskIm).polygon(polygon, outline=1, fill=1)
        mask = np.array(maskIm)
        
        # assemble new image (unit8: 0~255)
        newImArray = np.empty(imArray.shape, dtype='uint8')
        
        # colors (three first columns, RGB)
        newImArray[:,:,:3] = imArray[:,:,:3]
        
        # transparency (4th column)
        newImArray[:,:,3] = mask*255
        
        # back to Image from numpy
        newIm = Image.fromarray(newImArray, "RGBA")
        
        # crop
        left = min(x_lst)
        right = max(x_lst)
        top = min(y_lst)
        bottom = max(y_lst)
        
        newIm = newIm.crop((left, top, right, bottom))
        # newIm = newIm.convert("RGB")
        
        # save img & lst
        idx = cnt
        dst_path = os.path.join(dst_dir_path, "imgs_src", str(cnt).zfill(12) + img_ext)
        label = i['label']
        
        for i in error_char:
            label = str(label.replace(i,''))
        
        label = label.strip()
        
        if label == '': # label이 없는 것 패스
            continue
            
        # if label == "v/v": # v/v인 라벨은 4/4로 수정
        #     label = "4/4"
        
        tmp_data.append(idx)
        tmp_data.append(dst_path)
        tmp_data.append(label)
        
        data_lst.append(tmp_data)
        
        newIm.save(dst_path, "png")
        cnt += 1
        
# make csv
csv_path = os.path.join(dst_dir_path, 'data_src.csv')
f = open(csv_path, "w")
wr = csv.writer(f)
wr.writerow(['', 'filename', 'words'])
for i in tqdm(data_lst):
    wr.writerow(i)
    
# train_test_split
train_lst, tmp_lst = train_test_split(data_lst, test_size=0.2, shuffle=True, random_state=34)
test_lst, val_lst = train_test_split(tmp_lst, test_size=0.5, shuffle=True, random_state=34)
print(f"len(train_lst): {len(train_lst)}")
print(f"len(test_lst): {len(test_lst)}")
print(f"len(val_lst): {len(val_lst)}")

# save train_test_split_dataset
modes = {'train': train_lst, 'test': test_lst, 'val': val_lst}

for k, v in modes.items():
    mode = k
    target_lst = v
    
    csv_path = os.path.join(dst_dir_path, f"{mode}_data.csv")
    img_dir_path = os.path.join(dst_dir_path, mode)
    os.makedirs(img_dir_path, exist_ok=True)

    final_tmp_lst = []
    for idx, data in enumerate(tqdm(target_lst)):
        tmp_lst = []

        img_path = data[1]
        label = data[2]

        if label != '':
            dst_img_path = img_path.replace('imgs_src', mode)

            tmp_lst.append(idx)
            tmp_lst.append(dst_img_path)
            tmp_lst.append(label)

            shutil.copy(img_path, dst_img_path)
            final_tmp_lst.append(tmp_lst)

    f = open(csv_path, "w")
    wr = csv.writer(f)
    wr.writerow(['', 'filename', 'words'])
    for i in tqdm(final_tmp_lst):
        wr.writerow(i)

 91%|█████████▏| 20892/22886 [1:38:26<04:20,  7.65it/s]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
# recognition dataset 작업결과 확인
test_imgs = sorted(glob(os.path.join(dst_dir_path, "imgs_src", '*')))
print(f"crop된 이미지 갯수: {len(test_imgs)}")
# print(test_imgs[-1])
# display(Image.open(test_imgs[-1]))
    
print(len(data_lst))
print(data_lst[-1])
print('-----------------------------------------')
csv_file = os.path.join(dst_dir_path, 'data_src.csv')

with open(csv_file, "r")as f:
    for idx, data in enumerate(csv.reader(f)):
        if idx == 0:
            continue
        img_path = data[1]
        label = data[2]
        
        display(Image.open(img_path))
        print(label)
        if idx == 10:
            break

crop된 이미지 갯수: 879495
879495
[879494, '/home/ubuntu/workspace/seunghoon/z_dataset/diablo2_dataset_ver2/labelme_dataset_for_recognition/imgs_src/000000879494.png', '비교']
-----------------------------------------


블러드


그래스프


착용


가능한 레벨:


...

카오스큐브


명품관


신청합니다


+10%


매우빠른캐스팅


In [6]:
# make detection dataset

dst_dir_path = os.path.join(root_path, target_folder_name, "labelme_dataset_for_detection")
dst_img_dir_path = os.path.join(dst_dir_path, 'imgs_src')
dst_txt_dir_path = os.path.join(dst_dir_path, 'txts_src')
os.makedirs(dst_img_dir_path, exist_ok=True)
os.makedirs(dst_txt_dir_path, exist_ok=True)

for img in tqdm(src_imgs):
    # copy img
    img_dst_path = os.path.join(dst_img_dir_path, os.path.basename(img))
    shutil.copy(img, img_dst_path)
    
    # make txt file
    json_path = img.replace(os.path.splitext(img)[1], ".json")
    with open(json_path, "r")as f:
        data = json.load(f)
        
    final_label_lst = []
    for inform in data["shapes"]:
        tmp_label_lst = []
        for tmp_coors in inform["points"]:
            for tmp_coor in tmp_coors:
                tmp_label_lst.append(tmp_coor)
        tmp_label_lst.append(inform["label"])
        final_label_lst.append(tmp_label_lst)
        
    # print(final_label_lst)
    txt_path = img_dst_path.replace(os.path.splitext(img)[1], ".txt").replace("imgs_src", "txts_src")
    with open(txt_path, "w") as f:
        for label_line in final_label_lst:
            result = ""
            for t in label_line:
                result += str(t) + ","
            
            f.write(result[:-1] + "\n")

100%|██████████| 22886/22886 [00:38<00:00, 595.89it/s]
